<a href="https://colab.research.google.com/github/somkiatth/git/blob/master/kuRDD_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ทดลองลง **PySpark**

In [ ]:
print('Hello World!')

Hello World!


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 67.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9eee9c500d6991a4cfcdec0093c4cd0ee35a22e1d08d16a74e1a88a7052057f5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('rdd_demo').getOrCreate()

In [ ]:
coins = range(10)

In [ ]:
list(coins)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
coins_rdd = spark.sparkContext.parallelize(coins)

In [ ]:
coins_rdd.getNumPartitions()

2

In [ ]:
coin_squared_rdd = coins_rdd.map(lambda x: x**2)

In [ ]:
coin_squared_rdd.take(5)

[0, 1, 4, 9, 16]

In [ ]:
coin_squared_rdd.reduce(lambda x,y: x+y)

285

In [ ]:
import random

In [ ]:
flip_rdd = coins_rdd.map(lambda x: random.random() )

In [ ]:
flip_rdd.collect()

[0.08645653579670187,
 0.6442600470634781,
 0.6519859889233526,
 0.3017287343559987,
 0.5754198563574969,
 0.45944762573904174,
 0.7393389096700232,
 0.1955402874819705,
 0.45621425520771464,
 0.16793399939968257]

In [ ]:
head_rdd= flip_rdd.filter(lambda x: x > 0.5)

In [ ]:
head_rdd.collect()

[0.9029962119914815, 0.9678036555359747, 0.6130005704445302]

In [ ]:
flip_rdd_cached = coins_rdd.map(lambda x: random.random() ).cache()

In [ ]:
flip_rdd_cached.collect()

[0.6755736977235276,
 0.7034235451126585,
 0.29883780833951745,
 0.48666913566597625,
 0.2999132293612924,
 0.2370995490319946,
 0.8066286693847678,
 0.11684050370466281,
 0.31796131864956756,
 0.3006018493885453]

In [ ]:
flip_rdd_cached.filter(lambda x: x > 0.5).collect()

[0.6755736977235276, 0.7034235451126585, 0.8066286693847678]

In [ ]:
file_path= '/content/drive/MyDrive/Datasets/book-war-and-peace.txt'

In [ ]:
book_rdd = spark.sparkContext.textFile(file_path)

In [ ]:
book_rdd.take(10)

['CHAPTER I',
 '',
 '"Well, Prince, so Genoa and Lucca are now just family estates of the',
 "Buonapartes. But I warn you, if you don't tell me that this means war,",
 'if you still try to defend the infamies and horrors perpetrated by that',
 'Antichrist--I really believe he is Antichrist--I will have nothing more',
 "to do with you and you are no longer my friend, no longer my 'faithful",
 "slave,' as you call yourself! But how do you do? I see I have frightened",
 'you--sit down and tell me all the news."',
 '']

In [ ]:
book_rdd.map(lambda line: line.split() ).take(10)

[['CHAPTER', 'I'],
 [],
 ['"Well,',
  'Prince,',
  'so',
  'Genoa',
  'and',
  'Lucca',
  'are',
  'now',
  'just',
  'family',
  'estates',
  'of',
  'the'],
 ['Buonapartes.',
  'But',
  'I',
  'warn',
  'you,',
  'if',
  'you',
  "don't",
  'tell',
  'me',
  'that',
  'this',
  'means',
  'war,'],
 ['if',
  'you',
  'still',
  'try',
  'to',
  'defend',
  'the',
  'infamies',
  'and',
  'horrors',
  'perpetrated',
  'by',
  'that'],
 ['Antichrist--I',
  'really',
  'believe',
  'he',
  'is',
  'Antichrist--I',
  'will',
  'have',
  'nothing',
  'more'],
 ['to',
  'do',
  'with',
  'you',
  'and',
  'you',
  'are',
  'no',
  'longer',
  'my',
  'friend,',
  'no',
  'longer',
  'my',
  "'faithful"],
 ["slave,'",
  'as',
  'you',
  'call',
  'yourself!',
  'But',
  'how',
  'do',
  'you',
  'do?',
  'I',
  'see',
  'I',
  'have',
  'frightened'],
 ['you--sit', 'down', 'and', 'tell', 'me', 'all', 'the', 'news."'],
 []]

In [ ]:
words_rdd = book_rdd.flatMap(lambda line: line.split() ).cache()

In [ ]:
words_rdd.count()

562488

In [ ]:
book_rdd.count()

63845

In [ ]:
count_war = 0

In [ ]:
def add_war(x):
  global count_war
  if x == 'war':
    count_war = count_war + 1

In [ ]:
words_rdd.map(add_war).count()

562488

In [ ]:
count_war

0

In [ ]:
def tag_war(x):
  if x=="war":
    return 1
  else:
    return 0

In [ ]:
words_rdd.map(tag_war).reduce(lambda x,y: x+y)

166

In [ ]:
accum_war = spark.sparkContext.accumulator(0)

In [ ]:
def add_war_accum(x):
  if x=='war':
    accum_war.add(1)

In [ ]:
accum_war.value

0

In [ ]:
words_rdd.foreach(add_war_accum)

In [ ]:
accum_war.value

166

In [ ]:
words_kv_rdd = words_rdd.map(lambda x: (x,1) )

In [ ]:
words_kv_rdd.take(5)

[('CHAPTER', 1), ('I', 1), ('"Well,', 1), ('Prince,', 1), ('so', 1)]

In [ ]:
words_kv_rdd.groupByKey().take(5)

[('Prince,', <pyspark.resultiterable.ResultIterable at 0x7f4bada09650>),
 ('Genoa', <pyspark.resultiterable.ResultIterable at 0x7f4bada6bd50>),
 ('Lucca', <pyspark.resultiterable.ResultIterable at 0x7f4bada09c50>),
 ('are', <pyspark.resultiterable.ResultIterable at 0x7f4bada09510>),
 ('now', <pyspark.resultiterable.ResultIterable at 0x7f4bada09210>)]

In [ ]:
words_kv_rdd.groupByKey()\
                    .map(lambda x: ( x[0], sum(x[1]) ) )\
                    .take(5)

[('Prince,', 17), ('Genoa', 3), ('Lucca', 2), ('are', 1141), ('now', 873)]

In [ ]:
word_count_rdd = words_kv_rdd.reduceByKey(lambda x,y: x+y)

In [ ]:
word_count_rdd.sortByKey().take(10)

[('"\'Come', 1),
 ('"\'Dieu', 1),
 ('"\'Dio', 1),
 ('"\'From', 1),
 ('"\'Grant', 1),
 ('"\'I', 4),
 ('"\'No', 1),
 ('"\'Now', 1),
 ('"\'Russia', 1),
 ('"\'Sergey', 1)]

In [ ]:
word_count_rdd.sortBy(lambda x: x[1], ascending=False).take(20)

[('the', 31550),
 ('and', 20498),
 ('to', 16252),
 ('of', 14746),
 ('a', 9984),
 ('in', 8048),
 ('his', 7632),
 ('he', 7627),
 ('that', 7222),
 ('was', 7189),
 ('with', 5475),
 ('had', 5305),
 ('not', 4392),
 ('at', 4189),
 ('her', 3883),
 ('as', 3690),
 ('on', 3358),
 ('it', 3313),
 ('for', 3249),
 ('I', 3208)]

In [ ]:
print(word_count_rdd.toDebugString().decode('utf-8'))

(2) PythonRDD[73] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[51] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[50] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[49] at reduceByKey at <ipython-input-76-bcf73ace9622>:1 []
    |  PythonRDD[48] at reduceByKey at <ipython-input-76-bcf73ace9622>:1 []
    |  PythonRDD[16] at RDD at PythonRDD.scala:53 []
    |      CachedPartitions: 2; MemorySize: 2.4 MiB; DiskSize: 0.0 B
    |  /content/drive/MyDrive/Datasets/book-war-and-peace.txt MapPartitionsRDD[10] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /content/drive/MyDrive/Datasets/book-war-and-peace.txt HadoopRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
